In [13]:
import ollama


In [14]:
from langchain_ollama import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings
model = OllamaLLM(model="llama3.2")
embeddings = OllamaEmbeddings(model = "llama3.2")
model.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!"

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta."

In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'mlschool.pdf', 'page': 0}, page_content='Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine\nlearning systems from the ground up.\nNext cohort:\xa0February3 - 20, 2025\nCheck the schedulefor more details about upcoming cohorts.\nI want to join! Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and finding\nhelp to solve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models\nfor some of the largest companies in the world. I want to show you how to do the\nsame.\nThis is the class I wish I had taken when I started.\n"This is the best machine learning course I\'ve done.\nWorth every cent.

In [17]:
from langchain.prompts import PromptTemplate

template="""
Answer the question based on the context below. If you can't answer the question, reply "I don't know".
Context: {context}
Question: {question}
"""
prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't answer the question, reply "I don't know".
Context: Here is some context
Question: Here is a question



In [18]:
chain = prompt| model | parser

In [19]:
chain.input_schema.schema()

C:\Users\dhruv\AppData\Local\Temp\ipykernel_20224\3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [20]:
chain.invoke(
    {
    "context":"The name I was given is Dhruv",
    "question":"What's my name?"
}
)

'Your name is Dhruv.'

In [21]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding = embeddings
)


In [22]:
retriever = vectorstore.as_retriever()
retriever.invoke("Machine Learning")

[Document(metadata={'source': 'mlschool.pdf', 'page': 2}, page_content='This program is for software engineers,data scientists,data analysts,machine\nlearning engineers,technical managers, and anyone anyone who wants to use\nmachine learning to solve real-world problems.\nHere are the criteria to succeed in the program:\nYou have experience writing code. We\'ll use Python throughout the class,\nbut you won\'t have any problems if you know any other language.\nYou are familiar with basic machine learning terminology. This is not an\nintroductory class. We\'ll move quickly over the basics to focus on the\nfundamental ideas that make systems work.\nYou are ready to put in the work to succeed.\n"I have learned a ton from Santiago in his class and it was actually what helped inspire me\nand get into the MLOps work that I\'m doing now. Truly one of the most helpful online courses\nfor doing real, full-scale machine learning."\nBrian H. Hough\nSoftware Engineer\nWhat Will You Learn?\nYou\'ll 

In [23]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)